In [1]:
from kp_2d_benchmark import DATASET_DIR



In [2]:
URL = "https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?"

CUB_DATASET_DIR = DATASET_DIR / "CUB_200_2011"
print(CUB_DATASET_DIR)
if not CUB_DATASET_DIR.exists():
    print("oei")




/home/tlips/Code/2D-keypoints-benchmark/data/datasets/CUB_200_2011


In [3]:
image_id_file = CUB_DATASET_DIR / "images.txt"
part_id_file = CUB_DATASET_DIR / "parts/parts.txt"

part_annotations_file = CUB_DATASET_DIR / "parts/part_locs.txt"
bbox_annotations_file = CUB_DATASET_DIR / "bounding_boxes.txt"

train_test_split_file = CUB_DATASET_DIR / "train_test_split.txt"

In [4]:
image_id_to_path_dict = {}
with open(image_id_file, "r") as f:
    for line in f:
        image_id, image_path = line.strip().split()
        image_id_to_path_dict[int(image_id)] = image_path.strip("'")

part_id_to_name_dict = {}
with open(part_id_file, "r") as f:
    for line in f:
        part_id, part_name = line.strip().split()[0], "_".join(line.strip().split()[1:])

        part_id_to_name_dict[int(part_id)] = part_name

train_image_ids = []
test_image_ids = []
with open(train_test_split_file, "r") as f:
    for line in f:
        image_id, is_train = line.strip().split()
        if is_train == "1":
            train_image_ids.append(int(image_id))
        else:
            test_image_ids.append(int(image_id))

In [5]:
bbox_annotations = {}
with open(bbox_annotations_file, "r") as f:
    for line in f:
        image_id, x, y, width, height = line.strip().split()
        bbox_annotations[int(image_id)] = (float(x), float(y), float(width), float(height))


In [6]:
part_annotations = {}
with open(part_annotations_file, "r") as f:
    for line in f:
        image_id, part_id, x, y, visible = line.strip().split()
        image_id = int(image_id)
        part_id = int(part_id)
        x = float(x)
        y = float(y)
        visible = int(visible)

        if image_id not in part_annotations:
            part_annotations[image_id] = {}

        part_annotations[image_id][part_id] = (x, y, visible)

print(len(part_annotations))


11788


In [7]:
from kp_2d_benchmark.eval.coco_results import CocoKeypointAnnotation, CocoKeypointCategory, CocoImage, CocoKeypointsDataset, CocoInstancesDataset
from PIL import Image

In [8]:
category = CocoKeypointCategory(supercategory="animal", id=1, name="bird", keypoints = part_id_to_name_dict.values(), skeleton = [[i, i+1] for i in range(0, len(part_id_to_name_dict), 2)])

train_images, test_images = [], []
for image_id, image_path in image_id_to_path_dict.items():
    image_path =  f"images/{image_path}"
    # open image and get size
    with Image.open(CUB_DATASET_DIR / image_path) as img:
        w,h = img.size
    image = CocoImage(file_name=image_path, id=image_id,height=h,width=w)
    if image_id in train_image_ids:
        train_images.append(image)
    else:
        test_images.append(image)

train_annotations, test_annotations = [], []
for image_id, parts in part_annotations.items():
    keypoints = []
    for part_id, (x, y, visible) in parts.items():
        visible = 2 if visible == 1 else 0
        keypoints.extend([x, y, visible])
    img = Image.open(CUB_DATASET_DIR / "images" / image_id_to_path_dict[image_id])
    w,h = img.size

    if bbox_annotations[image_id][2] + bbox_annotations[image_id][0] > w:
        tup = bbox_annotations[image_id]
        bbox_annotations[image_id] =(tup[0],tup[1],w-tup[0]-2,tup[3])
    if bbox_annotations[image_id][3] + bbox_annotations[image_id][1] > h:
        tup = bbox_annotations[image_id]
        bbox_annotations[image_id] =(tup[0],tup[1],tup[2],h-tup[1]-2)
    bbox = bbox_annotations[image_id]

    annotation = CocoKeypointAnnotation(image_id=image_id,id=image_id, category_id=1, keypoints=keypoints, bbox=bbox,num_keypoints=sum(1 for x in keypoints[2::3] if x == 2))
    if image_id in train_image_ids:
        train_annotations.append(annotation)
    else:
        test_annotations.append(annotation)


test_dataset = CocoKeypointsDataset(images=test_images, annotations=test_annotations, categories=[category])
train_dataset = CocoKeypointsDataset(images=train_images, annotations=train_annotations, categories=[category])


In [9]:
print(len(train_dataset.annotations))
print(len(test_dataset.annotations))

5994
5794


In [10]:
# save to json
import json
with open(CUB_DATASET_DIR / "train_annotations.json", "w") as f:
    json.dump(train_dataset.model_dump(), f)

with open(CUB_DATASET_DIR / "test_annotations.json", "w") as f:
    json.dump(test_dataset.model_dump(), f)

In [11]:
# use split 
command = f" airo-dataset-tools split-coco-dataset {CUB_DATASET_DIR / 'train_annotations.json'} --split-ratio 0.9 --split-ratio 0.1"
import os 
os.system(command)

Split sizes: [5395, 599]


0

In [12]:
# resize all splits

command = f" airo-dataset-tools resize-coco-dataset {CUB_DATASET_DIR / 'train_annotations_train.json'} --width 512 --height 512"
os.system(command)



None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


100%|██████████| 5395/5395 [00:24<00:00, 219.65it/s]


0

In [13]:
command = f" airo-dataset-tools resize-coco-dataset {CUB_DATASET_DIR / 'test_annotations.json'} --width 512 --height 512"
os.system(command)



None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


100%|██████████| 5794/5794 [00:26<00:00, 220.73it/s]


0

In [14]:
command = f" airo-dataset-tools resize-coco-dataset {CUB_DATASET_DIR / 'train_annotations_val.json'} --width 512 --height 512"
os.system(command)


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


100%|██████████| 599/599 [00:02<00:00, 211.18it/s]


0

In [15]:
# make folder for resized datasets

resized_dir = CUB_DATASET_DIR / "512x512"

resized_dir.mkdir(exist_ok=True)

os.system(f"mv {CUB_DATASET_DIR.parent / "test_annotations_resized_512x512"} {resized_dir / 'test'}")
os.system(f"mv {CUB_DATASET_DIR.parent / "train_annotations_train_resized_512x512"} {resized_dir / 'train'}")
os.system(f"mv {CUB_DATASET_DIR.parent / "train_annotations_val_resized_512x512"} {resized_dir / 'val'}")

0